In [ ]:
#source: https://towardsdatascience.com/using-the-strava-api-and-pandas-to-explore-your-activity-data-d94901d9bfde

In [ ]:
import requests, urllib3, polyline, plotly.express as px, pandas as pd
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

#input the client_id, client_secret, and referesh_token
payload = {
    'client_id': "",
    'client_secret': '',
    'refresh_token': '',
    'grant_type': "refresh_token",
    'f': 'json'
}

#Since a token gets deleted this requests a new token
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

#This now requests your data
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

print(my_dataset[0]["name"])
print(my_dataset[0]["map"]["summary_polyline"])

# Converting polyline format to GPS for Tableau

In [ ]:
#Pull the upload_id (unique identifier) and map's polyline from each exercise session
map_data = {}
for i in range(len(my_dataset)):
    
    map_data[i] = [my_dataset[i]["upload_id"], my_dataset[i]['map']['summary_polyline']]
    
#Iterate over every polyline to convert to GPS coordinates
gps_map = {}

for i in range(len(map_data)):
    gps_map[i] = [map_data[i][0],polyline.decode(map_data[i][1])]
    
#Flatten the list of lists of just gps coordinates
gps_map_flat = []

for i in range(len(gps_map)):
    for x in range(len(gps_map[i][1])):
        gps_map_flat.append([gps_map[i][0],gps_map[i][1][x][0], gps_map[i][1][x][1]])
        
#Merge gps coordinates with extra data columns from each exercise sessions
gps_map = pd.DataFrame(gps_map_flat) #convert to dataframe
gps_map.columns = ['upload_id','lat','long'] #rename columns

#convert json to dataframe for all data attributes
activities = pd.json_normalize(my_dataset)

strava_final = gps_map.merge(activities, on = 'upload_id', how = 'left')

strava_final.to_csv(r'C:\Users\tadde\OneDrive\Python Scripts\strava_gps_final.csv', index = False)